In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Read the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Set initial tracking points
p0 = None
landmark_points = []

# Mouse callback function for initial point selection
def select_point(event, x, y, flags, param):
    global p0
    if event == cv2.EVENT_LBUTTONDOWN:
        p0 = np.array([[x, y]], dtype=np.float32)

# Create a window and bind the mouse callback function
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', select_point)

while True:
    # Read the current frame
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect landmarks with Mediapipe Pose
    results = pose.process(frame)
    if results.pose_landmarks:
        # Extract landmark points
        landmark_points = []
        for landmark in results.pose_landmarks.landmark:
            x = int(landmark.x * frame.shape[1])
            y = int(landmark.y * frame.shape[0])
            landmark_points.append((x, y))

        # Overlay Lucas-Kanade optical flow on landmarks
        if p0 is not None:
            # Calculate optical flow using Lucas-Kanade algorithm
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            # Select good points
            if p1 is not None:
                good_new = p1
                good_old = p0

                # Draw the Lucas-Kanade tracking points as green dots
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    a, b = new.ravel()
                    c, d = old.ravel()
                    frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)
                    frame = cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)

                # Update the previous frame and points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)

        # Draw light opacity landmarks on frame
        mp_drawing = mp.solutions.drawing_utils
        annotated_image = frame.copy()
        mp_drawing.draw_landmarks(
            annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2))

        # Display the frame with landmarks and optical flow
        cv2.imshow('Frame', annotated_image)

    else:
        # Display the original frame without landmarks
        cv2.imshow('Frame', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()
